O diretório de cotacoes, desenvolvido pelo prof. Geovany Borges, é utilizado para obter os dados da B3 organizados por carteira a partir das séries históricas fornecidas pelo site da B3 de 1986 até o presente.

O script atualizar_dados.py, responsável por gerar os dados ajustados de cada carteira. O procedimento leva muito tempo para ser processado. Foi implementada uma modificação neste código para reduzir o tempo de processamento, utilizando processos paralelos com a biblioteca multiprocessing. A otimização foi possível porque o gargalo de desempenho do programa é o uso elevado de cpu e o script, sem aplicação de paralelismo, utiliza apenas uma thread (ou cpu lógica) do sistema.

Os trechos modificados do código podem ser vistos abaixo.

In [1]:
def main():
    
    import time
    use_paralellism = True
    if use_paralellism:
        import multiprocessing
        import psutil
        
        #...
        
    if use_paralellism:
        cpu_count = psutil.cpu_count(logical=False)
        print("\n*** Processo utilizando paralelismo com", cpu_count, "processos!")
        lista_codigos = [codigos_da_carteira_b3_acoes_eod[i::cpu_count] for i in range(cpu_count)]
        processes = []
        for i in range(cpu_count):
            processes.append(multiprocessing.Process(target = atualizar_cotacoes_b3_acoes_eod, args = (lista_codigos[i],)))
            processes[-1].start()
        for i_process in processes:
            i_process.join()
    else:
        atualizar_cotacoes_b3_acoes_eod(codigos_da_carteira_b3_acoes_eod)
    
    elapsed_time = time.time() - start_time    
    print("\n*** Processo concluído em %d minutos e %f segundos!" %(elapsed_time/60, elapsed_time%60))

    return

Como é possível notar, é possível escolher se será feito o uso de paralelismo ou não a partir da variável booleana use_paralellism.

Caso não seja utilizado paralelismo (use_paralellism = False), o código irá rodar como antes e irá chamar a função atualizar_cotacoes_b3_acoes_eod() com a lista completa codigos_da_carteira_b3_acoes_eod, fazendo o processamento de cada elemento da lista na função de forma sequencial.

Caso seja utilizado paralelismo (use_paralellism = True), o código primeiramente irá obter a quantidade de núcleos físicos (cores) do sistema, armazenado na variável cpu_count, que será utilizada para dividir a lista codigos_da_carteira_b3_acoes_eod em uma lista de listas lista_codigos e determinar quantos processos. Esta lista de listas possui um número de listas igual ao número de núcleos físicos cpu_count, sendo que cada lista é uma parte da lista codigos_da_carteira_b3_acoes_eod contendo o número de elementos de codigos_da_carteira_b3_acoes_eod dividido por cpu_count elementos. Também é criada outra lista processes que ira armazenar cada processo rodando paralelamente. Em um loop for é adicionado na lista process cada um dos cpu_count processos, utilizando como argumento um elemento da lista de listas lista_codigos, e então este último processo adicionado é inicializado. Após instanciar e inicializar cada processo paralelo, é necessário fazer a parte principal do programa esperar o término de todos os processos para então prosseguir, o que é feito com o próximo loop for chamando a função join() para cada processo.

Com o término do processamento da função atualizar_cotacoes_b3_acoes_eod(), seja de forma sequencial ou paralela, o programa se encerra mostrando quanto tempo levou para ser concluído